In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

import logging
import pytorch_lightning as pl
import warnings

warnings.filterwarnings('ignore')
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)

## Data load

In [ ]:
! mkdir ../../data
! curl -OL https://storage.yandexcloud.net/di-datasets/age-prediction-nti-sbebank-2019.zip
! unzip -j -o age-prediction-nti-sbebank-2019.zip 'data/*.csv' -d ../../data
! mv age-prediction-nti-sbebank-2019.zip ../../data/

## Data Preproccessing

In [1]:
import os
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


data_path = '../../data/'

spark = SparkSession.builder.master("local[8]").appName("PysparkDataPreprocessor").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/11 18:17:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
source_data = spark.read.option("header",True).csv(os.path.join(data_path, 'transactions_train.csv'))
source_data.show(2)

+---------+----------+-----------+----------+
|client_id|trans_date|small_group|amount_rur|
+---------+----------+-----------+----------+
|    33172|         6|          4|    71.463|
|    33172|         6|         35|    45.017|
+---------+----------+-----------+----------+
only showing top 2 rows



In [ ]:
dir(source_data)

In [6]:
# pd.to_datetime(dt[self.cols_event_time]).min()

source_data.select(F.to_timestamp(source_data.trans_date).alias('dt')).show()   #.agg({'dt': 'min'}).collect()[0]['min(dt)']

+----+
|  dt|
+----+
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
+----+
only showing top 20 rows



In [ ]:
import logging

from .base import DataPreprocessor

logger = logging.getLogger(__name__)


class PysparkDataPreprocessor(DataPreprocessor):
    """Data preprocessor based on pandas.DataFrame
    During preprocessing it
        * transform `cols_event_time` column with date and time
        * encodes category columns `cols_category` into ints;
        * apply logarithm transformation to `cols_log_norm' columns;
        * groups flat data by `col_id`;
        * arranges data into list of dicts with features
    Parameters
    ----------
    col_id : str
        name of column with ids
    cols_event_time : str,
        name of column with time and date
    cols_category : list[str],s
        list of category columns
    cols_log_norm : list[str],
        list of columns to be logarithmed
    cols_identity : list[str],
        list of columns to be passed as is without any transformation
    cols_target: List[str],
        list of columns with target
    time_transformation: str. Default: 'default'.
        type of transformation to be applied to time column
    print_dataset_info : bool. Default: False.
        If True, print dataset stats during preprocessor fitting and data transformation
    """
    def __init__(self,
                 col_id: str,
                 cols_event_time: str,
                 cols_category: List[str],
                 cols_log_norm: List[str],
                 cols_identity: List[str],
                 cols_target: List[str] = [],
                 time_transformation: str = 'default'):

        super().__init__(col_id, cols_event_time, cols_category, cols_log_norm, cols_identity, cols_target)
        self.print_dataset_info = print_dataset_info
        self.time_transformation = time_transformation
        self.time_min = None
        
        
    def fit(self, dt, **params):
        """
        Parameters
        ----------
        dt : pandas.DataFrame with flat data
        Returns
        -------
        self : object
            Fitted preprocessor.
        """
        # Reset internal state before fitting
        self._reset()

        for col in self.cols_category:
            mapping = {row[col]: i + 1 for i, row in enumerate(dt.select(col).distinct().collect())}
            self.cols_category_mapping[col] = mapping

        for col in self.cols_log_norm:
            dt = dt.withColumn('sign', F.when(F.col(col) >= 0, 1).otherwise(-1))
            self.cols_log_norm_maxes[col] = source_data.select((F.log1p(F.abs(F.col(col))) * F.col('sign')).alias('log1p_signed'))\
                                                                    .agg({"log1p_signed": "max"}).collect()[0]['max(log1p_signed)']

        if self.time_transformation == 'hours_from_min':
            self.time_min = pd.to_datetime(dt[self.cols_event_time]).min()

        return self
    
    
    

    def _reset(self):
        """Reset internal data-dependent state of the preprocessor, if necessary.
        __init__ parameters are not touched.
        """
        # TODO: pyspark reset data-dependent state of the preprocessor
        pass

    def fit(self, dt, **params):
        """
        Parameters
        ----------
        dt : pandas.DataFrame with flat data
        Returns
        -------
        self : object
            Fitted preprocessor.
        """
        # Reset internal state before fitting
        self._reset()

        # TODO: pyspark fit

        return self

    def transform(self, dt, **params):
        self.check_is_fitted()

        # TODO: pyspark transformation

        return dt

    def check_is_fitted(self):
        # TODO: pyspark is preprocessor fitted check
        pass

In [8]:
import os
import pandas as pd
import numpy as np


data_path = '../../data/'

source_data_pd = pd.read_csv(os.path.join(data_path, 'transactions_train.csv'))
source_data_pd.head(2)

,client_id,trans_date,small_group,amount_rur
0,33172,6,4,71.463
1,33172,6,35,45.017


In [9]:
pd.to_datetime(source_data_pd['trans_date'])

0          1970-01-01 00:00:00.000000006
1          1970-01-01 00:00:00.000000006
2          1970-01-01 00:00:00.000000008
3          1970-01-01 00:00:00.000000009
4          1970-01-01 00:00:00.000000010
                        ...             
26450572   1970-01-01 00:00:00.000000727
26450573   1970-01-01 00:00:00.000000727
26450574   1970-01-01 00:00:00.000000727
26450575   1970-01-01 00:00:00.000000727
26450576   1970-01-01 00:00:00.000000729
Name: trans_date, Length: 26450577, dtype: datetime64[ns]

In [36]:
dt = source_data_pd

(np.log1p(abs(dt[col])) * np.sign(dt[col])).max()

12.899439277009005

In [5]:
from dltranz.data_preprocessing import PandasDataPreprocessor

preprocessor = PandasDataPreprocessor(
    col_id='client_id',
    cols_event_time='trans_date',
    time_transformation='float',
    cols_category=["trans_date", "small_group"],
    cols_log_norm=["amount_rur"],
    cols_identity=[],
    print_dataset_info=False,
)

In [6]:
%%time

dataset = preprocessor.fit_transform(source_data)

CPU times: user 50.4 s, sys: 3.2 s, total: 53.6 s
Wall time: 53.5 s


In [7]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dataset, test_size=0.2, random_state=42)

print(len(train), len(test))

24000 6000


## Embedding training

Model training in our framework organised via pytorch-lightning (pl) framework.
The key parts of neural networks training in pl are: 

    * model (pl.LightningModule)
    * data_module (pl.LightningDataModule)
    * pl.trainer (pl.trainer)
    
For futher details check https://www.pytorchlightning.ai/

### model 

In [8]:
from dltranz.seq_encoder import SequenceEncoder
from dltranz.models import Head
from dltranz.lightning_modules.emb_module import EmbModule

seq_encoder = SequenceEncoder(
    category_features=preprocessor.get_category_sizes(),
    numeric_features=["amount_rur"],
    trx_embedding_noize=0.003
)

head = Head(input_size=seq_encoder.embedding_size, use_norm_encoder=True)

model = EmbModule(seq_encoder=seq_encoder, head=head)

### Data module

In [9]:
from dltranz.data_load.data_module.emb_data_module import EmbeddingTrainDataModule

dm = EmbeddingTrainDataModule(
    dataset=train,
    pl_module=model,
    min_seq_len=25,
    seq_split_strategy='SampleSlices',
    category_names = model.seq_encoder.category_names,
    category_max_size = model.seq_encoder.category_max_size,
    split_count=5,
    split_cnt_min=25,
    split_cnt_max=200,
    train_num_workers=16,
    train_batch_size=256,
    valid_num_workers=16,
    valid_batch_size=256
)

### Trainer

In [10]:
import torch
import pytorch_lightning as pl

import logging
# logging.getLogger("lightning").addHandler(logging.NullHandler())
# logging.getLogger("lightning").propagate = False

trainer = pl.Trainer(
#     progress_bar_refresh_rate=0,
    max_epochs=150,
    gpus=1 if torch.cuda.is_available() else 0
)

### Training 

In [ ]:
%%time

trainer.fit(model, dm)

## Inference 

In [12]:
# embedding inference

from dltranz.inference import get_embeddings

train_embeds = get_embeddings(
    data=train,
    model=model, 
    category_names = model.seq_encoder.category_names,
    category_max_size = model.seq_encoder.category_max_size,
)

test_embeds = get_embeddings(
    data=test,
    model=model, 
    category_names = model.seq_encoder.category_names,
    category_max_size = model.seq_encoder.category_max_size,
)

train_embeds.shape, test_embeds.shape

24000it [00:01, 21719.33it/s]
6000it [00:00, 31278.41it/s]                                                                                                            


((24000, 512), (6000, 512))

In [13]:
# join target and embeddings

df_target = pd.read_csv(os.path.join(data_path, 'train_target.csv'))
df_target = df_target.set_index('client_id')
df_target.rename(columns={"bins": "target"}, inplace=True)

train_df = pd.DataFrame(data=train_embeds, columns=[f'embed_{i}' for i in range(train_embeds.shape[1])])
train_df['client_id'] = [x['client_id'] for x in train]
train_df = train_df.merge(df_target, how='left', on='client_id')

test_df = pd.DataFrame(data=test_embeds, columns=[f'embed_{i}' for i in range(test_embeds.shape[1])])
test_df['client_id'] = [x['client_id'] for x in test]
test_df = test_df.merge(df_target, how='left', on='client_id')

print(train_df.shape, test_df.shape)
train_df.head(2)

(24000, 514) (6000, 514)


,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9,...,embed_504,embed_505,embed_506,embed_507,embed_508,embed_509,embed_510,embed_511,client_id,target
0,0.344823,0.340152,0.231864,-0.789117,-0.013289,-0.056129,-0.988241,-0.010464,-0.064898,-0.029179,...,0.342959,0.040367,0.253053,0.712581,-0.148498,0.016645,-0.124844,-0.078120,36253,1
1,0.276711,0.492666,0.781279,-0.824952,0.020340,-0.014695,-0.891105,-0.047485,0.063514,0.170822,...,0.372329,0.049816,0.346733,0.465750,-0.080580,0.011563,-0.040877,-0.028394,396,2


Obtained embeddings can be used as features for model training

For example:

In [14]:
from sklearn.linear_model import LogisticRegression

embed_columns = [x for x in train_df.columns if x.startswith('embed')]
x_train, y_train = train_df[embed_columns], train_df['target']
x_test, y_test = test_df[embed_columns], test_df['target']

clf = LogisticRegression()
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

0.6296666666666667